In [32]:
import boto3
import os
import json
import sagemaker
from sagemaker.mxnet import MXNet

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [4]:
# Configura tu bucket
bucket_name = "my-image-classification-bucket-dog-cat"
s3 = boto3.client('s3')

# Si no existe, crear el bucket
s3_resource = boto3.resource('s3')
if not s3_resource.Bucket(bucket_name) in s3_resource.buckets.all():
    s3.create_bucket(Bucket=bucket_name)
print(f"bucket {bucket_name} creado")

bucket my-image-classification-bucket-dog-cat


In [5]:
def upload_folder_to_s3(local_folder, bucket, s3_folder):
    for root, dirs, files in os.walk(local_folder):
        for file in files:
            if file.endswith(('.jpg', '.png')):
                local_path = os.path.join(root, file)
                relative_path = os.path.relpath(local_path, local_folder)
                s3_path = f"{s3_folder}/{relative_path}"
                s3.upload_file(local_path, bucket, s3_path)
                print(f"Subido {local_path} -> s3://{bucket}/{s3_path}")

In [7]:
# Subir train y validation
# upload_folder_to_s3('dataset/train', bucket_name, 'train')
# upload_folder_to_s3('dataset/validation', bucket_name, 'validation')
print("Imagenes subidas")

Subido dataset/train/cats/cat.10005.jpg -> s3://my-image-classification-bucket-dog-cat/train/cats/cat.10005.jpg
Subido dataset/train/cats/cat.10127.jpg -> s3://my-image-classification-bucket-dog-cat/train/cats/cat.10127.jpg
Subido dataset/train/cats/cat.10034.jpg -> s3://my-image-classification-bucket-dog-cat/train/cats/cat.10034.jpg
Subido dataset/train/cats/cat.10020.jpg -> s3://my-image-classification-bucket-dog-cat/train/cats/cat.10020.jpg
Subido dataset/train/cats/cat.10046.jpg -> s3://my-image-classification-bucket-dog-cat/train/cats/cat.10046.jpg
Subido dataset/train/cats/cat.10093.jpg -> s3://my-image-classification-bucket-dog-cat/train/cats/cat.10093.jpg
Subido dataset/train/cats/cat.10089.jpg -> s3://my-image-classification-bucket-dog-cat/train/cats/cat.10089.jpg
Subido dataset/train/cats/cat.10116.jpg -> s3://my-image-classification-bucket-dog-cat/train/cats/cat.10116.jpg
Subido dataset/train/cats/cat.10145.jpg -> s3://my-image-classification-bucket-dog-cat/train/cats/cat.10

In [23]:
def create_manifest_from_s3(s3_prefix, output_file):
    """
    s3_prefix: carpeta en S3 (ej. 'train' o 'validation')
    output_file: nombre del archivo manifest a crear
    """
    manifest_lines = []

    # Listar objetos en S3
    paginator = s3.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=bucket_name, Prefix=s3_prefix):
        for obj in page.get('Contents', []):
            key = obj['Key']
            if key.endswith(('.jpg', '.png')):
                # Obtener clase desde la carpeta padre
                # Ej: train/cats/cat1.jpg -> cats
                class_name = key.split('/')[1]
                class_idx = 0 if class_name == 'cats' else 1  # ajusta según tus clases
                s3_path = f"s3://{bucket_name}/{key}"
                manifest_lines.append(json.dumps({
                    "source-ref": s3_path,
                    "class": str(class_idx)
                }))

    # Guardar manifest localmente
    with open(output_file, 'w') as f:
        f.write('\n'.join(manifest_lines))
    print(f"Manifest generado: {output_file}")

In [24]:
# Crear manifests desde S3
create_manifest_from_s3('dataset/train', 'train.manifest')
create_manifest_from_s3('dataset/validation', 'validation.manifest')

Manifest generado: train.manifest
Manifest generado: validation.manifest


In [26]:
manifest_file = 'train.manifest'  # o 'validation.manifest'

with open(manifest_file, 'r') as f:
    lines = f.readlines()

# Mostrar las primeras 10 líneas para inspeccionar
for line in lines[:10]:
    
    print(line)

{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.0.jpg", "class": "1"}

{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.1.jpg", "class": "1"}

{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.10.jpg", "class": "1"}

{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.100.jpg", "class": "1"}

{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.1000.jpg", "class": "1"}

{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.10000.jpg", "class": "1"}

{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.10001.jpg", "class": "1"}

{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.10002.jpg", "class": "1"}

{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.10003.jpg", "class": "1"}

{"source-ref": "s3://my-

In [27]:
s3.upload_file('train.manifest', bucket_name, 'manifests/train.manifest')
s3.upload_file('validation.manifest', bucket_name, 'manifests/validation.manifest')

print("Manifests subidos a S3")

Manifests subidos a S3


In [30]:

def read_manifest_from_s3(s3_bucket, s3_key, n_lines=10):
    """
    s3_bucket: nombre del bucket
    s3_key: ruta al manifest dentro del bucket
    n_lines: número de líneas a mostrar
    """
    obj = s3.get_object(Bucket=s3_bucket, Key=s3_key)
    lines = obj['Body'].read().decode('utf-8').splitlines()
    
    # Mostrar primeras n_lines
    print(f"Primeras {n_lines} líneas del manifest:")
    for line in lines[:n_lines]:
        print(line)
    

In [31]:
read_manifest_from_s3(bucket_name, 'manifests/train.manifest')


Primeras 10 líneas del manifest:
{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.0.jpg", "class": "1"}
{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.1.jpg", "class": "1"}
{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.10.jpg", "class": "1"}
{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.100.jpg", "class": "1"}
{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.1000.jpg", "class": "1"}
{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.10000.jpg", "class": "1"}
{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.10001.jpg", "class": "1"}
{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.10002.jpg", "class": "1"}
{"source-ref": "s3://my-image-classification-bucket-dog-cat/dataset/train/cats/cat.10003.jpg", "class": "1"}


In [59]:
# identificación de región
region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))
region = boto3.Session().region_name
# indetificación de rol
role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))
# sesión de sagemaker
sess  = sagemaker.Session()

AWS Region: us-east-1
RoleArn: arn:aws:iam::981520344224:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole


In [34]:
bucket_name = "my-image-classification-bucket"

train_manifest = f"s3://{bucket_name}/manifests/train.manifest"
validation_manifest = f"s3://{bucket_name}/manifests/validation.manifest"


In [56]:

estimator = MXNet(
    entry_point='train.py',
    role=role,
    framework_version='1.9.0',  # versión soportada
    py_version='py38',           # cambiar a py38
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    hyperparameters={
        'num_classes': 2,
        'num_layers': 18,
        'image_shape': '3,224,224',
        'batch_size': 32,
        'epochs': 10,
        'learning_rate': 0.01
    }
)


In [57]:
from sagemaker.inputs import TrainingInput

train_input = TrainingInput(
    s3_data=train_manifest,
    distribution='FullyReplicated',
    content_type='application/x-image',
    s3_data_type='ManifestFile'
)

validation_input = TrainingInput(
    s3_data=validation_manifest,
    distribution='FullyReplicated',
    content_type='application/x-image',
    s3_data_type='ManifestFile'
)

In [58]:
estimator.fit({'train': train_input, 'validation': validation_input})


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: mxnet-training-2025-10-17-16-04-06-510
ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-training-job


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 estimator.fit({'train': train_input, 'validation': validation_input})                        │
│   2                                                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/ │
│ telemetry_logging.py:168 in wrapper                                                              │
│                                                                                                  │
│   165 │   │   │   │   │   caught_ex = e                                                          │
│   166 │   │   │   │   finally:                                                                   │
│   167 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 168 │   │   │   │   │   │   raise caught_ex                                                    │
│   169 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   170 │   │   │   else:                                                                          │
│   171 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/ │
│ telemetry_logging.py:139 in wrapper                                                              │
│                                                                                                  │
│   136 │   │   │   │   start_timer = perf_counter()                                               │
│   137 │   │   │   │   try:                                                                       │
│   138 │   │   │   │   │   # Call the original function                                           │
│ ❱ 139 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   140 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   141 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   142 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/workflow/p │
│ ipeline_context.py:346 in wrapper                                                                │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.

In [47]:
from sagemaker import image_uris
region = sagemaker.Session().boto_region_name

image_uri = image_uris.retrieve(
    framework='image-classification',  # algoritmo integrado
    region=region,
    version='1',                        # versión del algoritmo
    py_version='py3',                   # Python 3
    instance_type='ml.p3.2xlarge'
)
print("Contenedor del algoritmo integrado:", image_uri)


Contenedor del algoritmo integrado: 811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1


In [54]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput

bucket_name = "my-image-classification-bucket"

# Entradas de datos
train_input = TrainingInput(
    s3_data=f"s3://{bucket_name}/manifests/train.manifest",
    content_type='application/x-image',
    s3_data_type='ManifestFile'
)

validation_input = TrainingInput(
    s3_data=f"s3://{bucket_name}/manifests/validation.manifest",
    content_type='application/x-image',
    s3_data_type='ManifestFile'
)

# Estimator integrado
ic_estimator = Estimator(
    image_uri=image_uri,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.g4dn.xlarge',  # cambiar a una instancia con cuota
    output_path=f's3://{bucket_name}/output',
    hyperparameters={
        'num_classes': 2,
        'image_shape': '3,224,224',
        'num_layers': 18,
        'epochs': 10,
        'batch_size': 32,
        'learning_rate': 0.01
    }
)


In [55]:
ic_estimator.fit({'train': train_input, 'validation': validation_input})


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: image-classification-2025-10-17-16-03-05-588
ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-training-job


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 ic_estimator.fit({'train': train_input, 'validation': validation_input})                     │
│   2                                                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/ │
│ telemetry_logging.py:168 in wrapper                                                              │
│                                                                                                  │
│   165 │   │   │   │   │   caught_ex = e                                                          │
│   166 │   │   │   │   finally:                                                                   │
│   167 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 168 │   │   │   │   │   │   raise caught_ex                                                    │
│   169 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   170 │   │   │   else:                                                                          │
│   171 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/ │
│ telemetry_logging.py:139 in wrapper                                                              │
│                                                                                                  │
│   136 │   │   │   │   start_timer = perf_counter()                                               │
│   137 │   │   │   │   try:                                                                       │
│   138 │   │   │   │   │   # Call the original function                                           │
│ ❱ 139 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   140 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   141 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   142 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/workflow/p │
│ ipeline_context.py:346 in wrapper                                                                │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.